<a href="https://colab.research.google.com/github/santoshapatil/Edx-All-Courses/blob/main/edx_all_Courses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt update
!apt install chromium-chromedriver
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results


In [59]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd


In [ ]:
wd = 
wd.get("https://www.edx.org/search")
print(wd.page_source)  # results
# divs = wd.find_elements_by_css_selector('div')

In [1]:
url="https://www.edx.org/search?subject=Architecture&tab=course"

In [60]:
course_name=[]
course_type=[]
course_photo_URL=[]
typ=[]
typ_name=[]
university_name=[]
university_logo=[]
course_link=[]
course_tit=[]
time_required=[]
course_level=[]
course_language=[]
course_subtitles=[]
course_skills=[]
category=[]
sub_category=[]
course_rating=[]

driver = webdriver.Chrome(options=options)
driver.implicitly_wait(5)
driver.get(url)

#calculating number of pages 
page_str = driver.find_element_by_xpath('(//button[@class="btn page-link"])[last()]').text
total_page = int(page_str)



In [ ]:
total_page

In [16]:
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(5)
driver.get(url)
page = driver.page_source
soup = BeautifulSoup(page, "html.parser")


In [79]:
page_str = driver.find_element_by_xpath('(//button[@class="btn page-link"])[last()]').text
total_page = int(page_str)
total_page

2

In [ ]:
butt.click()

In [ ]:
course_name=[]
course_type=[]
course_photo_URL=[]
typ=[]
typ_name=[]
university_name=[]
university_logo=[]
course_link=[]
course_tit=[]
time_required=[]
course_level=[]
course_language=[]
course_subtitles=[]
course_skills=[]
category=[]
sub_category=[]
course_rating=[]



urls={"https://www.edx.org/search?subject=Architecture&tab=course","https://www.edx.org/search?subject=Food%20%26%20Nutrition&tab=course"}
for u in urls:
  driver = webdriver.Chrome(options=options)
  driver.implicitly_wait(2)
  driver.get(u)
  n_of_pgs = driver.find_element_by_xpath('(//button[@class="btn page-link"])[last()]').text
  n_of_pg = int(n_of_pgs)
  get_course(u,n_of_pg)

In [96]:
len(course_link)

72

In [ ]:
course_name

In [101]:
def get_course(u,n_p):

  total_page=n_p
  driver = webdriver.Chrome(options=options)
  driver.implicitly_wait(2)
  driver.get(u)
  page = driver.page_source
  soup = BeautifulSoup(page, "html.parser")
  for i in range(1,total_page+1):
  #driver = webdriver.Chrome(options=options)
  #driver.implicitly_wait(5)
  
    go=i
    c_xpath = '//div[@class="discovery-card Verified and Audit col col-xl-3 mb-4 static-card-spacing"]/a[@class="discovery-card-link"]'
    c_p_pg=len(soup.find_all('a',class_='discovery-card-link'))
    print(c_p_pg)
    for j in range(c_p_pg):
    
    #Attaching course Name/Title
      cnn=soup.find_all('h3',class_="name-heading")[j]
      if cnn is None:
        continue
      else:
        cnn=cnn.get_text()
        course_name.append(cnn)
    
    #Attaching Course Link
      c_url=soup.find_all('a',class_='discovery-card-link')[j]
      clink="https://www.edx.org" + c_url.get('href')
      course_link.append(clink)

    #attaching University /name
      univ = soup.find_all('div',class_='provider')[j]
      university_name.append(univ)

    #attaching University photo
      try:
        u_logo=soup.find('img',class_='partner-logo')[j]
        l=u_logo.get('src')
        university_logo.append(l)
      except:
        university_logo.append("not-mentioned")
    
    #attaching Course photo
    
      try:

        c_pic=soup.find_all('div',class_='d-card-hero')[j]
        cp=c_pic.find('img').get('src')
        course_photo_URL.append(cp)
      except:
        course_photo_URL.append("not-mentioned")
    if i!=total_page:
      driver.find_element_by_xpath('//button[@class="btn next page-link"]').click()
    
      WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, c_xpath)))
    #wait for another 30 seconds for the page to load completely
      time.sleep(10)
      #driver.get(url)
      go=go+1
      check = '//button[@aria-label="Page ' + str(go) + ', Current Page"]'
      page = driver.page_source
      soup = BeautifulSoup(page, "html.parser")
    else:
      break
  print("done")

In [75]:
course_name=[]
course_type=[]
course_photo_URL=[]
typ=[]
typ_name=[]
university_name=[]
university_logo=[]
course_link=[]
course_tit=[]
time_required=[]
course_level=[]
course_language=[]
course_subtitles=[]
course_skills=[]
category=[]
sub_category=[]
course_rating=[]
for i in range(1,total_page+1):
  #driver = webdriver.Chrome(options=options)
  #driver.implicitly_wait(5)
  
  go=1
  c_xpath = '//div[@class="discovery-card Verified and Audit col col-xl-3 mb-4 static-card-spacing"]/a[@class="discovery-card-link"]'
  c_p_pg=len(soup.find_all('a',class_='discovery-card-link'))
  for j in range(c_p_pg):
    
    #Attaching course Name/Title
    cnn=soup.find_all('h3',class_="name-heading")[j]
    if cnn is None:
      continue
    else:
      cnn=cnn.get_text()
      course_name.append(cnn)
    
    #Attaching Course Link
    c_url=soup.find_all('a',class_='discovery-card-link')[j]
    clink="https://www.edx.org" + c_url.get('href')
    course_link.append(clink)

    #attaching University /name
    univ = soup.find_all('div',class_='provider')[j]
    university_name.append(univ)

    #attaching University photo
    try:
      u_logo=soup.find('img',class_='partner-logo')[j]
      l=u_logo.get('src')
      university_logo.append(l)
    except:
      university_logo.append("not-mentioned")
    
    #attaching Course photo
    
    try:

      c_pic=soup.find_all('div',class_='d-card-hero')[j]
      cp=c_pic.find('img').get('src')
      course_photo_URL.append(cp)
    except:
      course_photo_URL.append("not-mentioned")
  if i!=total_page:
    driver.find_element_by_xpath('//button[@class="btn next page-link"]').click()
    
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, c_xpath)))
    #wait for another 30 seconds for the page to load completely
    time.sleep(10)
    #driver.get(url)
    go=go+1
    check = '//button[@aria-label="Page ' + str(go) + ', Current Page"]'
    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")
  else:
    break


    
    








In [89]:
course_name

['Science & Cooking: FromHaute Cuisine to SoftMatter Science (chemis……',
 'The Health Effects ofClimate Change…',
 'Nutrition, Exercise andSports…',
 'FlashDiet: Aprende acomer sano…',
 'Nutrition and Health:Macronutrients andOvernutrition…',
 'Science & Cooking: FromHaute Cuisine to SoftMatter Science (physics)…',
 'The Science of Beer…',
 'World of Wine: FromGrape to Glass…',
 'Nutrition and Health:Micronutrients andMalnutrition…',
 'Nutrition and Health:Human Microbiome…',
 'Nutrition, Heart Diseaseand Diabetes…',
 'Mi propia huerta orgánica…',
 'Nutrition and Cancer…',
 'Beer: the science ofbrewing…',
 'Nutrition and Health:Food Risks…',
 'UnderstandingAgribusiness, ValueChains, and Consumers……',
 'Sustainable FoodSecurity: The value ofsystems thinking…',
 'Bienestar para eldesarrollo personal yprofesional…',
 'Lifestyle MedicineFundamentals…',
 'Feeding a Hungry Planet:Agriculture, Nutrition andSustainability…',
 'Lifestyle MedicineTreatments of ChronicDisease – Part 1…',
 'Sustai

In [ ]:
course_name=[]
for j in range(c_p_pg):
  #Attaching course Name/Title
  cnn=soup.find_all('h3',class_="name-heading")[j]
  cnn=cnn.get_text()
  course_name.append(cnn)
  if i<=tot_pg:
    driver.find_element_by_xpath('//button[@class="btn next page-link"]').click()
    WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.XPATH, c_xpath)))
    #wait for another 30 seconds for the page to load completely
    time.sleep(30)
    driver.get(url)
    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")

In [92]:
len(course_name)

96

In [ ]:
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(5)
driver.get(url)
page = driver.page_source
soup = BeautifulSoup(page, "html.parser")